In [1]:
PORT = 8080
#store url of peer nodes
peer_nodes = [8070,8080,8090,9000]
peer_nodes.remove(PORT)


In [2]:
from flask import Flask #flask is similar to django
from flask import request
import json
import requests
import hashlib
import datetime as date
from requests.exceptions import ConnectionError

In [3]:
node = Flask(__name__)

In [4]:
class Block:
    def __init__(self,index,timestamp,data,nonce,previous_hash,cur_hash):
        
        self.index = index
        self.timestamp = str(timestamp)
        self.data = data
        self.nonce = nonce
        self.previous_hash = str(previous_hash)
        self.hash = str(cur_hash) 
        
    def hash_block(self):
        print("In hash()")
        print(self.index,self.timestamp,self.data,self.previous_hash,self.nonce,self.hash)
        #all the data to hash upon, note: it needs to be encoded to utf-8
        prehash = (str(self.index)+str(self.timestamp)+str(self.data)+str(self.previous_hash)+str(self.nonce)).encode('utf-8') 
        result = hashlib.sha256(prehash)
        return str(result.hexdigest())

In [5]:
address = "user2_addr"

In [6]:
blockchain = []
pending_transactions = []

In [7]:
def create_transaction(peer_addr,amt):
    global blockchain
    global pending_transactions
    pending_transactions.append({"from":address,"to":peer_addr,"amt":amt})

In [8]:
@node.route('/getPendingTransactions', methods=['GET'])
def sendPendingTransactions():
    global blockchain
    global pending_transactions
    
    print(pending_transactions)
    pending_transactions_json = json.dumps(pending_transactions)
    pending_transactions = []
    return pending_transactions_json

In [9]:
def get_chain_json():
    global blockchain
    global pending_transactions
    chain_to_send = []
    #convert blocks into dictionaries so that they can be sent as json objects
    for block in blockchain:
        chain_to_send.append({
            "index": block.index,
            "timestamp": block.timestamp,
            "data" : block.data,
            "nonce" : block.nonce,
            "previous_hash" : block.previous_hash,
            "hash" : block.hash
        })
    
    #convert the list of dicts into json object
    chain_to_send = json.dumps(chain_to_send)
    return chain_to_send

In [10]:
@node.route('/getchain', methods=['GET'])
def send_chain():
    global blockchain
    global pending_transactions
    
    
    
    print("chain sent successfully")
    return  get_chain_json()

In [11]:
#check if computed hash on each block is equal to hash stored in it
def verify_chain(chain):
    global blockchain
    global pending_transactions
    for block in chain[1:]:
        if str(block.hash_block()) != block.hash:
            print("computed hash:",block.hash_block())
            print("hash found:",block.hash)
            print("chain Invalid at block index",block.index)
            return False
    return True

In [12]:
#obtain new longest chain from peers using their localhost urls
@node.route('/updateChain', methods=['GET'])
def find_new_longest_chain():
    global blockchain
    global pending_transactions
    for peer_port in peer_nodes:
        node_url = "http://localhost:" + str(peer_port)
        peer_chain_json = requests.get(node_url + "/getchain").content
        peer_chain_dict = json.loads(peer_chain_json)
        
        peer_chain = []
        for b in peer_chain_dict:
            block = Block(b["index"],b["timestamp"],b["data"],b["nonce"],b["previous_hash"],b["hash"])
            peer_chain.append(block)
        
        if verify_chain(peer_chain) and len(peer_chain) > len(blockchain):
            print("Blockchain updated")
            blockchain = peer_chain
        
            
    print(blockchain)
    return get_chain_json()

In [13]:
@node.route('/txion', methods=['POST'])
def transaction():
    global blockchain
    global pending_transactions
    #a json object is sent here containing details of the new transaction
    new_transaction = request.get_json()
#     new_transaction = json.loads(new_transaction)
    new_transaction["from"] = address
    pending_transactions.append(new_transaction)
    print(pending_transactions)
#     print("New transaction")
#     print("From:",new_transaction["from"].encode('ascii','replace'))
#     print("To:",new_transaction["to"].encode('ascii','replace'))
#     print("Amount:",new_transaction["amount"])
    
    #now send the client who sent request a msg
    return "Transaction added to pending transaction\n"

In [14]:
node.run(host='0.0.0.0', port=PORT, threaded=True, debug=False)

# while(True):
#     inp = input("1 for create transaction, 2 for view chain")
#     if inp == "1":
#         peer_addr = input("peer_addr: ")
#         amt = input("amount: ")
#         print(peer_addr,amt)
#         create_transaction(peer_addr,amt)
#         print("Pending Transactions",pending_transactions)
#     else:
#         find_new_longest_chain()
#         print(blockchain)
        

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2020 09:23:52] "GET /getchain HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 09:23:52] "GET /getPendingTransactions HTTP/1.1" 200 -


chain sent successfully
[]


127.0.0.1 - - [03/Nov/2020 09:24:02] "GET /getchain HTTP/1.1" 200 -


chain sent successfully


127.0.0.1 - - [03/Nov/2020 09:24:08] "GET /updateChain HTTP/1.1" 200 -


In hash()
1 2020-11-03 09:23:52.639956 [[{'to': 'e', 'amount': 10, 'from': 'user1_addr'}, {'to': 'f', 'amount': 10, 'from': 'user1_addr'}, {'to': 'g', 'amount': 10, 'from': 'user1_addr'}], [], []] c64b958390bebecf84e44327b226b6fd36f4c01b4e240dc5af377d315ad299eb 135 0008968996be1d1750e3e2a486ee9e3fc0145a4669e600a65d67c201f03603de
Blockchain updated
In hash()
1 2020-11-03 09:23:52.639956 [[{'to': 'e', 'amount': 10, 'from': 'user1_addr'}, {'to': 'f', 'amount': 10, 'from': 'user1_addr'}, {'to': 'g', 'amount': 10, 'from': 'user1_addr'}], [], []] c64b958390bebecf84e44327b226b6fd36f4c01b4e240dc5af377d315ad299eb 135 0008968996be1d1750e3e2a486ee9e3fc0145a4669e600a65d67c201f03603de
[<__main__.Block object at 0x7fbf67ea2c88>, <__main__.Block object at 0x7fbf67ea2c50>]
